In [17]:
import requests, time, random
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.expected_conditions import visibility_of_element_located
from selenium.webdriver.support.ui import WebDriverWait
import pyautogui
import keyboard
import glob
import os 
import pandas as pd
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from pandas.core.common import SettingWithCopyWarning
from IPython.display import display
import numpy as np

### read in data 

In [12]:
df1 = pd.read_csv("./data/University of Michigan-Ann Arbor.csv")
df2 = pd.read_csv("./data/University of Michigan-Ann Arbor2.csv")
frames = [df1, df2]
individual_profile = pd.concat(frames)

In [24]:
## set school name 
school_name = "University of Michigan-Ann Arbor"

In [14]:
## drop duplicates 
individual_profile = individual_profile.drop_duplicates(subset=['current_work'], keep='last')
individual_profile

,Unnamed: 0,current_location,latest_Education,current_work,previous_work,majro,search_school
1,1,Washington DC-Baltimore Area,University of Michigan - Ann Arbor,Resident at Urban Teachers,Religious School Teacher at Beth Israel Congre...,NaN,University of Michigan-Ann Arbor
2,2,"Swartz Creek, Michigan, United States",Baker College,Neurophysiologist at SpecialtyCare,Neurodiagnostic Technologist at MidMichigan He...,Movement Science,University of Michigan-Ann Arbor
3,3,"Ojai, California, United States",Northwestern University,Senior Analyst at Decision Resources Group,Market Analyst at Decision Resources Group,Journalism,University of Michigan-Ann Arbor
4,4,"Washington, District of Columbia, United States",University of Michigan - Ann Arbor,Lead Content Creator at Health Law Institute,Policy Intern at US Global Leadership Coalition,NaN,University of Michigan-Ann Arbor
6,6,"Vienna, Vienna, Austria",Fachhochschule Technikum Wien,Business & Integration Arch Associate at Accen...,Business Analyst at s IT Solutions,NaN,University of Michigan-Ann Arbor
...,...,...,...,...,...,...,...
881,881,Buffalo-Niagara Falls Area,University of Michigan,Division Chief Kidney and Pancreas Transplanta...,NaN,Clinical Outcomes and Statistics,University of Michigan-Ann Arbor
882,882,Detroit Metropolitan Area,University of Michigan - Ann Arbor,Google Associate Partner Brand Manager via Hug...,Cultural Brand Strategist at Fiverr Freelancer...,Sport and Fitness Administration/Management,University of Michigan-Ann Arbor
883,883,Detroit Metropolitan Area,University of Michigan - Ann Arbor,AME Director - US & Canada at Lear Corporation,Engineering Group Manager AME US & Canada at L...,Industrial Engineering,University of Michigan-Ann Arbor
884,884,"Vadodara, Gujarat, India",Rutgers University,Researcher PHD Student at Rutgers University,Advanced Manufacturing Engineer- Industry 4.0 ...,Mechanical Engineering,University of Michigan-Ann Arbor


## matching industry
### 1. get current and previuos company name

In [15]:
individual_profile.loc[:,'current_company'] = individual_profile['current_work'].str.split('at ').str[-1]
individual_profile.loc[:,'previous_company'] = individual_profile['previous_work'].str.split('at ').str[-1]

In [18]:
individual_profile.rename(columns={ individual_profile.columns[0]: "index" }, inplace = True)
## add new column
individual_profile['current_company_industry'] = np.nan
#pd.set_option('display.max_rows', 500)
individual_profile.head(5)

,index,current_location,latest_Education,current_work,previous_work,majro,search_school,current_company,previous_company,current_company_industry
1,1,Washington DC-Baltimore Area,University of Michigan - Ann Arbor,Resident at Urban Teachers,Religious School Teacher at Beth Israel Congre...,NaN,University of Michigan-Ann Arbor,Urban Teachers,Beth Israel Congregation - Ann Arbor,NaN
2,2,"Swartz Creek, Michigan, United States",Baker College,Neurophysiologist at SpecialtyCare,Neurodiagnostic Technologist at MidMichigan He...,Movement Science,University of Michigan-Ann Arbor,SpecialtyCare,MidMichigan Health,NaN
3,3,"Ojai, California, United States",Northwestern University,Senior Analyst at Decision Resources Group,Market Analyst at Decision Resources Group,Journalism,University of Michigan-Ann Arbor,Decision Resources Group,Decision Resources Group,NaN
4,4,"Washington, District of Columbia, United States",University of Michigan - Ann Arbor,Lead Content Creator at Health Law Institute,Policy Intern at US Global Leadership Coalition,NaN,University of Michigan-Ann Arbor,Health Law Institute,US Global Leadership Coalition,NaN
6,6,"Vienna, Vienna, Austria",Fachhochschule Technikum Wien,Business & Integration Arch Associate at Accen...,Business Analyst at s IT Solutions,NaN,University of Michigan-Ann Arbor,Accenture,s IT Solutions,NaN


In [9]:
## oepn browser 
browser = webdriver.Chrome('/Users/jarvis/Desktop/winter 22/CS for SS2/fianl project/final-project-ocean-s-4/chromedriver')
browser.get('https://web-p-ebscohost-com.proxy.uchicago.edu/ehost/search/advanced?vid=13&sid=a64a2a8c-e647-4836-8d08-8ad530f8d210%40redis')

<ipython-input-9-a48ea40b4e9f>:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome('/Users/jarvis/Desktop/winter 22/CS for SS2/fianl project/final-project-ocean-s-4/chromedriver')


### current company industry code 

In [ ]:
## input search term 
code_lst = []
for index, com in individual_profile.iterrows():
    index, search_term = index,com['current_company']
    browser.find_element(By.ID,"SearchTerm1").clear()
    search_box = browser.find_element(By.ID,"SearchTerm1")
    search_box.send_keys(search_term)
    search_box.send_keys(Keys.RETURN)
    ## click on result 
    try:
        table = browser.find_element(By.XPATH,"//*[@id='companyResults']/tbody")
        rows = table.find_element(By.XPATH,"//*[@id='companyResults']/tbody/tr[2]")
        rows.find_element(By.XPATH,'//*[@id="companyResults"]/tbody/tr[2]/td[2]/a').click()
        src = browser.page_source
        soup = BeautifulSoup(src, 'lxml')
        industry_info = soup.find_all('dl', {'class': 'data-details definition-list'})
        try:
            for ic in industry_info[2].find_all('a',{'data-auto':'link'}):
                individual_profile.loc[index,'current_company_industry'] = ic.text
        except IndexError:
            pass
    except NoSuchElementException:
        pass
   
    
    
    
    


In [22]:
# pd.set_option('display.max_rows', 500)
# individual_profile.head(15)
individual_profile.head(20)

,index,current_location,latest_Education,current_work,previous_work,majro,search_school,current_company,previous_company,current_company_industry
1,1,Washington DC-Baltimore Area,University of Michigan - Ann Arbor,Resident at Urban Teachers,Religious School Teacher at Beth Israel Congre...,NaN,University of Michigan-Ann Arbor,Urban Teachers,Beth Israel Congregation - Ann Arbor,NaN
2,2,"Swartz Creek, Michigan, United States",Baker College,Neurophysiologist at SpecialtyCare,Neurodiagnostic Technologist at MidMichigan He...,Movement Science,University of Michigan-Ann Arbor,SpecialtyCare,MidMichigan Health,621999 Stop smoking clinics
3,3,"Ojai, California, United States",Northwestern University,Senior Analyst at Decision Resources Group,Market Analyst at Decision Resources Group,Journalism,University of Michigan-Ann Arbor,Decision Resources Group,Decision Resources Group,551112 Public utility holding companies
4,4,"Washington, District of Columbia, United States",University of Michigan - Ann Arbor,Lead Content Creator at Health Law Institute,Policy Intern at US Global Leadership Coalition,NaN,University of Michigan-Ann Arbor,Health Law Institute,US Global Leadership Coalition,NaN
6,6,"Vienna, Vienna, Austria",Fachhochschule Technikum Wien,Business & Integration Arch Associate at Accen...,Business Analyst at s IT Solutions,NaN,University of Michigan-Ann Arbor,Accenture,s IT Solutions,541330 Traffic engineering consulting services
7,7,"Ann Arbor, Michigan, United States",University of Michigan - Ann Arbor,Business Development Student Associate at Stew...,Marketing and Analytics Intern at Elite Footba...,NaN,University of Michigan-Ann Arbor,"Stewbowie, LLC",Elite Football Network,NaN
8,8,Detroit Metropolitan Area,University of Michigan - Ann Arbor,President’s Club Mortgage Banker at Quicken Loans,NaN,Economics,University of Michigan-Ann Arbor,Quicken Loans,NaN,522292 Reverse mortgage lending
10,10,Detroit Metropolitan Area,University of Michigan - Ann Arbor,Labor and Delivery Nurse at Henry Ford Health ...,Labor and Delivery Nurse at Henry Ford Health ...,NaN,University of Michigan-Ann Arbor,Henry Ford Health System,Henry Ford Health System,622110 Osteopathic hospitals
11,11,"Caldwell, New Jersey, United States","Hunter College, School of Social Work, New Yor...",New Jersey Early Intervention Provider at self...,Behaviorist at Morris Plains School District,NaN,University of Michigan-Ann Arbor,self-employed,Morris Plains School District,813910 Wholesalers' associations
12,12,Detroit Metropolitan Area,University of Michigan - Stephen M. Ross Schoo...,Financial Cloud Applications Consultant at Oracle,Content Creator at Yaffe Digital Media Initiative,Psychology,University of Michigan-Ann Arbor,Oracle,Yaffe Digital Media Initiative,"511210 Utility software, computer, packaged"


In [26]:
filename = str(school_name)+"_with_industry_code.csv"
individual_profile.to_csv(filename,index = True)